# Begin Poly Art Algorithm: 

Written by: Anish Thite while procrastination

In [12]:
import numpy as np
import cv2
import sys
import numpy
import random
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
numpy.set_printoptions(threshold=sys.maxsize)

img = cv2.imread('gatech.jpg')

In [34]:
#RUNTIME COUNTER: n 
MINDIM = min(img.shape[:-1])
NUMVERTECES = (MINDIM**2) // 30
#NUMVERTECES = 10
print(NUMVERTECES)

7776


In [35]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    # return the edged image
    return edged

In [36]:
#Generate random points

img = cv2.imread('gatech.jpg')
img = cv2.resize(img,(MINDIM, MINDIM))
points = []
for _ in range(NUMVERTECES):
    points.append(np.random.randint(1,MINDIM,(1, 2)).tolist()[0])
corners = [[0,0],[0,MINDIM],[MINDIM,0],[MINDIM,MINDIM]]
points += corners


canny = auto_canny(img, sigma=0.1)


# img_grey = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
# ret,thresh = cv2.threshold(img_grey,200,255,0)
# contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
# cv2.drawContours(img, contours, -1, (0,255,0), 3)
# points = contours
#contours.reshape(-1,1)

nn = NearestNeighbors(2).fit(points)
n = nn.kneighbors(points, n_neighbors =5, return_distance=False)[:,1:]
triangles = np.hstack((n, np.reshape(np.arange(len(points)), (len(points),1))))
triangles = np.take(points, triangles, axis = 0)
for x in tqdm(triangles):
    #mean color
    #print('here')
    mask = np.zeros(img.shape, dtype=np.uint8)
    channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,)*channel_count
    cv2.fillConvexPoly(mask, np.int32([x]), ignore_mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    
    red = masked_img[:,:,0]
    green = masked_img[:,:,1]
    blue = masked_img[:,:,2]    
    cv2.fillConvexPoly(img, np.int32([x]), color=(np.mean(red[red != 0]),np.mean(green[green != 0]),np.mean(blue[blue != 0])))

    #cv2.polylines(img, np.int32([points]), isClosed=True, color=(0, 0, 255), thickness=1)

cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 600,600)
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

100%|██████████| 7780/7780 [00:07<00:00, 1099.33it/s]


In [ ]:
#make matrix for triangles and cube to get triangles:
graph = np.zeros((MINDIM, MINDIM))
for point in points:
    graph[point[0]][point[1]] = 1
    
np.dot(graph,(np.dot(graph,graph)))



